<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/CLAUDE3_AOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anthropic -q

!pip install colab-env --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import anthropic
import os
import colab_env
import json

Mounted at /content/gdrive


In [4]:
api_key = os.environ["CLAUDE3_API_KEY"]
model="claude-3-opus-20240229"

client = anthropic.Anthropic(
    api_key=api_key,
)

In [11]:
def claude_chat(prompt,model,client):
    message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": prompt}
    ]
    )
    print()
    print("-" * 80)
    print('Question: %s'%prompt)
    print("-" * 80)
    print()
    print('Answer: ')
    print(message.content[0].text)
    #return message.content[0].text

In [6]:
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude"}
    ]
)

In [ ]:
!pip install -q faiss-gpu -q

In [13]:
import colab_env
import os
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT


# Get the API key from the environment variable
api_key = os.environ.get("CLAUDE3_API_KEY")

# Initialize the Anthropic client with the retrieved API key
anthropic = Anthropic(api_key=api_key)

import random
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# --- External Data Simulation ---

def get_flight_data(flight_id):
    """Fetches flight data (simulated API call)."""
    storm_active = random.choice([True, False])
    flight_data = {
        "flight_id": flight_id,
        "aircraft_id": "AA123",
        "origin": "Paris",
        "destination": "London",
        "status": "en-route",
        "altitude": 35000,
        "speed": 500,
        "latitude": 50.0,
        "longitude": 2.0,
        "weather": {
            "temperature": -50,
            "wind": {
                "speed": 20,
                "direction": "West"
            },
            "storm": {
                "active": storm_active,
                "severity": "severe" if storm_active else "none",
                "location": {
                    "latitude": 51.0,
                    "longitude": 1.0
                }
            }
        }
    }
    return flight_data

def get_maintenance_records(aircraft_id):
    """Fetches maintenance records (simulated DB call)."""
    maintenance_records = [
        {
            "date": "2024-12-15",
            "description": "Scheduled engine inspection",
            "status": "completed"
        },
        {
            "date": "2025-01-05",
            "description": "Replaced hydraulic pump",
            "status": "completed"
        }
    ]
    return maintenance_records

def suggest_rerouting(flight_data):
    """Suggests rerouting based on weather."""
    if flight_data["weather"]["storm"]["active"]:
        suggestion = f"Consider rerouting flight {flight_data['flight_id']} to avoid the storm."
    else:
        suggestion = "No rerouting suggestions at this time."
    return suggestion

# --- FAISS Semantic Search Setup ---

# Sample documents (replace with your actual knowledge base)
documents = [
    "Aircraft maintenance is crucial for flight safety.",
    "Severe weather can cause flight delays and disruptions.",
    "Rerouting options should consider fuel efficiency and passenger comfort.",
    "In case of emergency, pilots should follow established procedures.",
    "Communication between pilots and air traffic control is essential.",
    "Regular training ensures crew members are prepared for various situations."
]

# Initialize SentenceTransformer for embeddings
encoder = SentenceTransformer('all-mpnet-base-v2')
embeddings = encoder.encode(documents)

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype('float32'))

def semantic_search(query, index, k=2):
    """Performs semantic search."""
    query_embedding = encoder.encode([query])[0].astype('float32')
    D, I = index.search(np.array([query_embedding]), k)
    return [documents[i] for i in I[0]]

# --- Cache-Augmented Generation ---

def preload_knowledge(query, index, k=2):
    """Preloads knowledge into the LLM context."""
    results = semantic_search(query, index, k)
    context = " ".join(results)
    return context

# --- AI Agent ---



def generate_response(prompt,model,client):
    message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": prompt}
    ]
    )
    print()
    print("-" * 80)
    print('Question: %s'%prompt)
    print("-" * 80)
    print()
    print('Answer: ')
    print(message.content[0].text)
    return message.content[0].text



def run_agent(question, index):
    """Executes the AI agent with Claude 3."""
    context = preload_knowledge(question, index)
    prompt = f"""You are an AI agent assisting in an airline operation context.
        Relevant context: {context}
        Available tools:
        * get_flight_data(flight_id): Get flight information.
        * suggest_rerouting(flight_data): Suggest rerouting.
        * get_maintenance_records(aircraft_id): Get maintenance records.
        Instructions:
        1. **Always** think first, then **clearly** state your thoughts.
        2. Choose the **best** tool action. Use 'Action Input' **only** if needed.
        3. **Always** give an 'Observation' after each action.
           Give a 'Final Answer' when you have all the info.
        4. Use this format:
           Question: ...
           Thought: ...
           Action: ...
           Action Input: ... (if needed)
           Observation: ...
        Begin!
        Question: {question}"""

    while True:
        #response = generate_response(prompt)
        response = generate_response(prompt,model,client)
        if "Final Answer:" in response:
            return response.split("Final Answer:")[-1].strip()

        try:
            action_line = next((line for line in response.split("\n") if line.startswith("Action: ")), None)
            input_line = next((line for line in response.split("\n") if line.startswith("Action Input: ")), None)

            if action_line and input_line:
                action = action_line.split("Action:")[-1].strip()
                action_input = input_line.split("Action Input:")[-1].strip()

                if action == "get_flight_data":
                    flight_data = get_flight_data(action_input)
                    observation = f"Flight data: {flight_data}"
                    try:
                        aircraft_id = flight_data["aircraft_id"]
                    except KeyError:
                        aircraft_id = "unknown"
                elif action == "suggest_rerouting":
                    flight_data = get_flight_data(action_input)
                    observation = suggest_rerouting(flight_data)
                elif action == "get_maintenance_records":
                    if action_input == "not provided":
                        observation = get_maintenance_records(aircraft_id)
                    else:
                        observation = get_maintenance_records(action_input)
                else:
                    observation = "Invalid action."
                prompt += f"\nObservation: {observation}"
            else:
                return "I'm sorry, I couldn't understand the model's response"
        except IndexError:
            return "I'm sorry, I couldn't understand the model's response"

# --- User Interaction ---

def get_user_input():
    """Simulates user input with a flight ID."""
    flight_id = "AA" + str(random.randint(1000, 9999))
    user_input = f"""
    Flight {flight_id} needs an update on their route, weather,
    and aircraft maintenance records. Provide info and suggest actions."""
    return user_input

# --- Main Execution ---

# Example usage (ensure FAISS index is passed to run_agent)
user_question = get_user_input()
response = run_agent(user_question, index)
print(response)


--------------------------------------------------------------------------------
Question: You are an AI agent assisting in an airline operation context.
        Relevant context: Severe weather can cause flight delays and disruptions. Aircraft maintenance is crucial for flight safety.
        Available tools:
        * get_flight_data(flight_id): Get flight information.
        * suggest_rerouting(flight_data): Suggest rerouting.  
        * get_maintenance_records(aircraft_id): Get maintenance records.
        Instructions:
        1. **Always** think first, then **clearly** state your thoughts.
        2. Choose the **best** tool action. Use 'Action Input' **only** if needed.
        3. **Always** give an 'Observation' after each action.
           Give a 'Final Answer' when you have all the info.
        4. Use this format:
           Question: ...
           Thought: ...
           Action: ...
           Action Input: ... (if needed)
           Observation: ...
        Begin!
   